# 模型训练

模型训练一般分为四个步骤：

1. 构建数据集。
2. 定义神经网络模型。
3. 定义超参、损失函数及优化器。
4. 输入数据集进行训练与评估。

现在我们有了数据集和模型后，可以进行模型的训练与评估。

## 构建数据集

首先从[数据集 Dataset](https://www.mindspore.cn/tutorials/zh-CN/r2.3/beginner/dataset.html)加载代码，构建数据集。

In [1]:
import mindspore
from mindspore import nn
from mindspore.dataset import vision, transforms
from mindspore.dataset import MnistDataset

# Download data from open datasets
from download import download

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/MNIST_Data.zip"
path = download(url, "./", kind="zip", replace=True)


def datapipe(path, batch_size):
    image_transforms = [
        vision.Rescale(1.0 / 255.0, 0),
        vision.Normalize(mean=(0.1307,), std=(0.3081,)),
        vision.HWC2CHW()
    ]
    label_transform = transforms.TypeCast(mindspore.int32)

    dataset = MnistDataset(path)
    dataset = dataset.map(image_transforms, 'image')
    dataset = dataset.map(label_transform, 'label')
    dataset = dataset.batch(batch_size)
    return dataset

train_dataset = datapipe('MNIST_Data/train', batch_size=64)
test_dataset = datapipe('MNIST_Data/test', batch_size=64)


file_sizes: 100%|██████████████████████████| 10.8M/10.8M [00:00<00:00, 13.8MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


## 定义神经网络模型

从[网络构建](https://www.mindspore.cn/tutorials/zh-CN/r2.3/beginner/model.html)中加载代码，构建一个神经网络模型。

In [2]:
class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_relu_sequential = nn.SequentialCell(
            nn.Dense(28*28, 512),
            nn.ReLU(),
            nn.Dense(512, 512),
            nn.ReLU(),
            nn.Dense(512, 10)
        )

    def construct(self, x):
        x = self.flatten(x)
        logits = self.dense_relu_sequential(x)
        return logits

model = Network()

## 定义超参、损失函数和优化器

### 超参

超参（Hyperparameters）是可以调整的参数，可以控制模型训练优化的过程，不同的超参数值可能会影响模型训练和收敛速度。

- **训练轮次（epoch）**：训练时遍历数据集的次数。

- **批次大小（batch size）**：数据集进行分批读取训练，设定每个批次数据的大小。batch size过小，花费时间多，同时梯度震荡严重，不利于收敛；batch size过大，不同batch的梯度方向没有任何变化，容易陷入局部极小值。

- **学习率（learning rate）**：如果学习率偏小，会导致收敛的速度变慢，如果学习率偏大，则可能会导致训练不收敛等不可预测的结果。

In [6]:
epochs = 3
batch_size = 64
learning_rate = 1e-2

### 损失函数

常见的损失函数包括用于回归任务的`nn.MSELoss`（均方误差）和用于分类的`nn.CrossEntropyLoss`（它结合了`nn.LogSoftmax`和`nn.NLLLoss`，可以对logits 进行归一化并计算预测误差）。

In [7]:
loss_fn = nn.CrossEntropyLoss()

### 优化器（Optimizer）

优化器内部定义了模型的参数优化过程（即梯度如何更新至模型参数），所有优化逻辑都封装在优化器对象中。

在这里，我们使用SGD（Stochastic Gradient Descent）优化器举例。

我们通过`model.trainable_params()`方法获得模型的可训练参数，并传入学习率超参来初始化优化器。

In [8]:
optimizer = nn.SGD(model.trainable_params(), learning_rate=learning_rate)

> 训练过程中，通过grad_fn计算获得参数对应的梯度，将其传入优化器中即可实现参数优化，具体形态如下：
>
> grads = grad_fn(inputs)
>
> optimizer(grads)


## 训练与评估

设置了超参、损失函数和优化器后，我们就可以循环输入数据来训练模型。一次数据集的完整迭代循环称为一轮（epoch）。每轮执行训练时包括两个步骤：

1. 训练：迭代训练数据集，并尝试收敛到最佳参数。
2. 验证/测试：迭代测试数据集，以检查模型性能是否提升。

接下来我们定义用于训练的`train_loop`函数和用于测试的`test_loop`函数。

- 先定义正向函数`forward_fn`，使用`value_and_grad`获得微分函数`grad_fn`。
- 然后，将微分函数和优化器的执行封装为`train_step`函数，接下来循环迭代数据集进行训练.

In [9]:
# 通过模型计算预测值，通过损失函数计算损失，并返回损失和预测值
def forward_fn(data, label):
    logits = model(data)
    loss = loss_fn(logits, label)
    return loss, logits

# 同时计算损失和梯度，None不指定特定参数位置，使用第三个参数中的参数列表，has_aux表示只对第一个元素（损失）计算梯度
grad_fn = mindspore.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

# 计算损失和梯度
def train_step(data, label):
    (loss, _), grads = grad_fn(data, label) #(loss, _)中的_表示忽略forward_fn返回的第二个值（logits）
    optimizer(grads)  #应用SGD优化器，对梯度更新模型参数
    return loss

def train_loop(model, dataset):
    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)  #对每个批次调用train_step进行训练

        if batch % 100 == 0:  #每100个批次打印一次损失信息
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")

`test_loop`函数同样需循环遍历数据集，调用模型计算loss和Accuray并返回最终结果。

In [10]:
def test_loop(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    #样本总量、积累测试过程中的损失、记录模型正确预测的样本数量
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label).asnumpy().sum()
    test_loss /= num_batches  #平均损失
    correct /= total        #准确率
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

我们将实例化的损失函数和优化器传入`train_loop`和`test_loop`中。训练3轮并输出loss和Accuracy，查看性能变化。

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = nn.SGD(model.trainable_params(), learning_rate=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model, train_dataset)
    test_loop(model, test_dataset, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.292444  [  0/938]
loss: 1.643636  [100/938]
loss: 0.952843  [200/938]
loss: 0.652146  [300/938]
loss: 0.433956  [400/938]
loss: 0.583112  [500/938]
loss: 0.254717  [600/938]
loss: 0.343774  [700/938]
loss: 0.348839  [800/938]
loss: 0.574244  [900/938]
Test: 
 Accuracy: 91.0%, Avg loss: 0.319791 

Epoch 2
-------------------------------
loss: 0.578773  [  0/938]
loss: 0.184648  [100/938]
loss: 0.277254  [200/938]
loss: 0.300438  [300/938]
loss: 0.237972  [400/938]
loss: 0.239813  [500/938]
loss: 0.196423  [600/938]
loss: 0.285938  [700/938]
loss: 0.164663  [800/938]
loss: 0.465331  [900/938]
Test: 
 Accuracy: 92.8%, Avg loss: 0.253174 

Epoch 3
-------------------------------
loss: 0.146422  [  0/938]
loss: 0.299450  [100/938]
loss: 0.323904  [200/938]
loss: 0.192631  [300/938]
loss: 0.243253  [400/938]
loss: 0.140615  [500/938]
loss: 0.408576  [600/938]
loss: 0.186512  [700/938]
loss: 0.202737  [800/938]
loss: 0.240933  [900/938]
Test: 
 